You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [13]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [14]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo' 

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [15]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [16]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [17]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [18]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [19]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [20]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [21]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-12-29 16:28:03,422 - 8517288512 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [22]:
result = crew.kickoff(inputs={"topic": "Electric Vehicles"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Electric Vehicles.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Electric Vehicles

Outline:
I. Introduction
- Brief overview of the rise of electric vehicles (EVs)
- Importance of transitioning to EVs for a sustainable future

II. Latest Trends in Electric Vehicles
- Growth in EV sales worldwide
- Advancements in battery technology
- Expansion of charging infrastructure

III. Key Players in the Electric Vehicle Industry
- Tesla
- Nissan
- Chevrolet
- BMW
- Ford

IV. Noteworthy News in Electric Vehicles
- Government incentives for EV purchases
- Partnerships betwee

- Display the results of your execution as markdown in the notebook.

In [23]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Electric Vehicles: Paving the Way for a Sustainable Future

In recent years, the popularity and adoption of electric vehicles (EVs) have seen a significant rise as the world transitions towards more sustainable transportation options. This shift plays a crucial role in mitigating the effects of climate change and reducing our dependence on fossil fuels. The transition to EVs for a sustainable future is vital, as they provide a cleaner and greener alternative to traditional gas-powered vehicles.

## Latest Trends in Electric Vehicles

The latest trends in the electric vehicle industry indicate a promising future for EVs. There has been a noticeable growth in EV sales globally, propelled by advancements in battery technology that have enhanced the range and efficiency of electric vehicles. Moreover, there has been a substantial expansion of charging infrastructure, making it more convenient and accessible for EV owners to recharge their vehicles while on the go.

## Key Players in the Electric Vehicle Industry

Several key players have emerged as leaders in the electric vehicle industry, with companies like Tesla, Nissan, Chevrolet, BMW, and Ford at the forefront of innovation in electric vehicle technology. These companies offer a variety of EV models to meet different consumer needs and preferences.

## Noteworthy News in Electric Vehicles

Recent news highlights government incentives for EV purchases, which have encouraged more consumers to make the switch to electric vehicles. Partnerships between automakers and tech companies for EV development have also resulted in exciting advancements in the electric vehicle industry. Consumer reviews and ratings of popular EV models provide valuable insights for potential buyers considering investing in an electric vehicle.

## Target Audience Analysis

The target audience for electric vehicles includes millennials and Gen Z individuals interested in eco-friendly transportation options, urban dwellers seeking cost-effective and convenient commuting solutions, and environmental enthusiasts focused on reducing carbon emissions. By catering to these diverse groups, the electric vehicle industry can continue to expand and thrive in the years to come.

In conclusion, electric vehicles are leading the way towards a more sustainable future by offering a cleaner and greener alternative to traditional gas-powered vehicles. By examining the latest trends, key players, and noteworthy news in the electric vehicle industry, consumers can make informed decisions about transitioning to EVs for a greener future. It is time to embrace the advantages of electric vehicles and consider switching to a more sustainable mode of transportation for a greener future.

**Call to Action:** 
I urge readers to explore the benefits of electric vehicles and consider transitioning to a more sustainable mode of transportation for a cleaner and greener future. Let's drive towards a cleaner and greener future with electric vehicles! 

Resources:
- [Electric Vehicle Association of America](https://electricdrive.org/)
- [Electric Cars Report](https://electriccarsreport.com/)
- [Green Car Reports](https://www.greencarreports.com/)
- [CleanTechnica](https://cleantechnica.com/)
- [EV Obsession](https://evobsession.com/)

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).